In [1]:
!pip install -q earthengine-api huggingface_hub pandas numpy pillow requests tqdm

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='tprotidtitititiiti')
print("GEE ready")
# import os
# os.environ['HF_TOKEN'] = 'hftoekn'

GEE ready


In [ ]:
import os
import time
import json
import requests
import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import tqdm
import ee

from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

session = requests.Session()
retry = Retry(total=5, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
adapter = HTTPAdapter(max_retries=retry)
session.mount("http://", adapter)
session.mount("https://", adapter)

BASE = Path("/content/data")
TILE_DIR = BASE / "tiles"
NDVI_DIR = BASE / "ndvi"
META_DIR = BASE / "meta"
for d in [BASE, TILE_DIR, NDVI_DIR, META_DIR]:
    d.mkdir(parents=True, exist_ok=True)

REGIONS = [
    (34.2, -118.0), (38.9, -77.0), (48.8, 2.35), (51.5, -0.12),
    (35.6, 139.6), (-33.86, 151.2), (-1.2921, 36.8219),
    (-23.5505, -46.6333), (19.4326, -99.1332), (55.75, 37.61)
]

DEBUG = False  # for quick test trueeeeee

@lru_cache(maxsize=None)
def fetch_open_meteo(lat: float, lon: float):
    """Never returns None. Always returns dict with floats."""
    url = (
        f"https://api.open-meteo.com/v1/forecast?"
        f"latitude={lat:.4f}&longitude={lon:.4f}"
        f"&daily=precipitation_sum,temperature_2m_max"
        f"&forecast_days=16&timezone=UTC"
    )
    for attempt in range(3):
        try:
            r = session.get(url, timeout=15)
            r.raise_for_status()
            data = r.json()
            daily = data.get("daily", {})
            precip = daily.get("precipitation_sum", [])
            temp = daily.get("temperature_2m_max", [])
            rain16 = sum(precip) if precip else 0.0
            temp16 = np.mean(temp) if temp and len(temp) > 0 else 15.0
            return {"rain16": float(rain16), "temp16": float(temp16)}
        except Exception as e:
            if attempt == 2:
                print(f"Meteo fallback used for {lat:.2f},{lon:.2f}")
                return {"rain16": 0.0, "temp16": 15.0}
            time.sleep(2 ** attempt)

def safe_download(url, path):
    try:
        r = session.get(url, timeout=30)
        if r.status_code == 200:
            path.write_bytes(r.content)
            return True
    except:
        pass
    return False

def fetch_tile_ndvi_srtm(lat, lon, tile_id, size=128):
    center = ee.Geometry.Point([lon, lat])
    try:
        s2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
              .filterBounds(center)
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
              .filterDate('2020-01-01', '2024-12-31')
              .sort('CLOUDY_PIXEL_PERCENTAGE')
              .first())
        if s2 is None:
            return False, None
    except:
        return False, None

    rgb = s2.select(['B4','B3','B2']).visualize(min=0, max=3000)
    ndvi = s2.normalizedDifference(['B8','B4']).rename('NDVI')
    srtm = ee.Image('USGS/SRTMGL1_003')
    region = center.buffer(1000).bounds()

    try:
        rgb_url = rgb.getThumbURL({'region': region, 'dimensions': f'{size}x{size}', 'format': 'png'})
        ndvi_url = ndvi.getThumbURL({'region': region, 'dimensions': f'{size}x{size}',
                                     'min': -0.5, 'max': 1.0,
                                     'palette': ['white', 'yellow', 'green']})
        srtm_url = srtm.getThumbURL({'region': region, 'dimensions': f'{size}x{size}',
                                     'min': 0, 'max': 3000, 'format': 'png'})
    except:
        return False, None

    ok1 = safe_download(rgb_url, TILE_DIR / f"{tile_id}.png")
    ok2 = safe_download(ndvi_url, NDVI_DIR / f"{tile_id}_ndvi.png")
    ok3 = safe_download(srtm_url, META_DIR / f"{tile_id}_srtm.png")
    return (ok1 and ok2 and ok3), None

def compute_soil_moisture_proxy(ndvi_mean, rain16):
    ndvi_mean = float(ndvi_mean or 0.3)
    rain16 = float(rain16 or 0.0)
    return float(np.clip(0.3 * ndvi_mean + 0.004 * rain16, 0.0, 1.0))

def compute_risk(elev_mean, ndvi_mean, rain16, soil_moisture):
    elev_mean = float(elev_mean or 100.0)
    ndvi_mean = float(ndvi_mean or 0.3)
    rain16 = float(rain16 or 0.0)
    soil_moisture = float(soil_moisture or 0.5)
    flood = (1.0 - elev_mean / 3000.0) * (rain16 / (rain16 + 50.0))
    wildfire = (1.0 - soil_moisture) * (1.0 - ndvi_mean)
    risk = 0.6 * flood + 0.4 * wildfire
    return float(np.clip(risk, 0.0, 1.0))

def process_tile(i):
    r_idx = i % len(REGIONS)
    base_lat, base_lon = REGIONS[r_idx]
    lat = base_lat + np.random.uniform(-0.4, 0.4)
    lon = base_lon + np.random.uniform(-0.6, 0.6)
    tile_id = f"tile_{i:06d}"

    ok, _ = fetch_tile_ndvi_srtm(lat, lon, tile_id, size=128)
    if not ok:
        return None

    try:
        ndvi_mean = np.mean(np.array(Image.open(NDVI_DIR / f"{tile_id}_ndvi.png").convert("L"))) / 255.0
    except:
        ndvi_mean = 0.3

    try:
        elev_mean = np.mean(np.array(Image.open(META_DIR / f"{tile_id}_srtm.png").convert("L"))) / 255.0 * 3000.0
    except:
        elev_mean = 100.0

    meteo = fetch_open_meteo(lat, lon)
    rain16 = meteo["rain16"]
    soil_moisture = compute_soil_moisture_proxy(ndvi_mean, rain16)
    risk = compute_risk(elev_mean, ndvi_mean, rain16, soil_moisture)

    return {
        "tile_id": tile_id,
        "lat": round(lat, 6),
        "lon": round(lon, 6),
        "elev_mean": float(elev_mean),
        "ndvi_mean": float(ndvi_mean),
        "rain16": float(rain16),
        "soil_moisture": float(soil_moisture),
        "risk": float(risk),
        "rgb_path": f"tiles/{tile_id}.png",
        "ndvi_path": f"ndvi/{tile_id}_ndvi.png",
        "srtm_path": f"meta/{tile_id}_srtm.png"
    }

def build_dataset(n_tiles=1200):
    if DEBUG:
        n_tiles = 10
    rows = []
    with ThreadPoolExecutor(max_workers=8) as ex:
        futures = [ex.submit(process_tile, i) for i in range(n_tiles)]
        for f in tqdm(as_completed(futures), total=n_tiles, desc="Building"):
            result = f.result()
            if result:
                rows.append(result)
    df = pd.DataFrame(rows)
    csv_path = BASE / "labels.csv"
    df.to_csv(csv_path, index=False)
    print(f"Dataset complete: {len(df)} tiles → {csv_path}")
    return df

df = build_dataset(n_tiles=1200)
df.head()

Building:   0%|          | 0/1200 [00:00<?, ?it/s]

Meteo fallback used for 19.78,-99.26
Meteo fallback used for 38.87,-76.89
Meteo fallback used for 38.89,-76.44
Meteo fallback used for 35.36,139.30
Meteo fallback used for 48.94,2.34
Meteo fallback used for 48.93,2.18
Meteo fallback used for 34.34,-118.59
Meteo fallback used for -33.94,151.33
Meteo fallback used for 38.99,-76.66
Meteo fallback used for -33.68,151.53
Meteo fallback used for 34.22,-117.96
Meteo fallback used for -23.32,-46.94
Meteo fallback used for 51.12,0.15
Meteo fallback used for -23.38,-46.40
Meteo fallback used for 35.41,139.07
Meteo fallback used for -1.66,37.03
Meteo fallback used for 51.48,-0.30
Meteo fallback used for 55.68,37.63
Meteo fallback used for -33.61,151.37
Meteo fallback used for 19.31,-99.50
Meteo fallback used for -1.06,37.21


Meteo fallback used for 34.52,-118.32
Meteo fallback used for 49.02,2.67
Meteo fallback used for -23.88,-46.31
Meteo fallback used for 39.25,-77.58
Meteo fallback used for -1.62,36.63
Meteo fallback used for 55.64,37.85
Meteo fallback used for 19.16,-99.06
Meteo fallback used for 34.42,-117.70
Meteo fallback used for 35.31,139.83
Meteo fallback used for 48.90,2.74
Meteo fallback used for 34.35,-118.54
Meteo fallback used for 19.75,-99.42
Meteo fallback used for 55.95,37.11
Meteo fallback used for 51.60,0.42
Meteo fallback used for 48.53,2.60
Meteo fallback used for 51.29,0.19
Meteo fallback used for 35.23,140.13
Meteo fallback used for 39.04,-76.48
Meteo fallback used for 51.83,-0.38
Meteo fallback used for 38.58,-76.80
Meteo fallback used for -34.19,150.85
Meteo fallback used for -33.69,151.15
Meteo fallback used for -23.90,-46.93
Meteo fallback used for 19.64,-98.96
Meteo fallback used for 35.58,139.11
Meteo fallback used for 55.74,37.71
Meteo fallback used for 33.96,-117.51
Meteo fa

Meteo fallback used for 19.35,-98.74
Meteo fallback used for -1.26,37.21
Meteo fallback used for 34.46,-118.06
Meteo fallback used for 34.40,-118.59
Meteo fallback used for 19.47,-99.62
Meteo fallback used for -23.26,-47.05
Meteo fallback used for 51.37,0.40
Meteo fallback used for 35.99,139.19
Meteo fallback used for 55.76,37.07
Meteo fallback used for 34.59,-117.86
Meteo fallback used for 39.20,-76.41
Meteo fallback used for 39.15,-77.57
Meteo fallback used for 48.54,1.81
Meteo fallback used for 48.63,2.08
Meteo fallback used for 38.75,-76.49
Meteo fallback used for 49.05,2.15
Meteo fallback used for -33.72,151.51
Meteo fallback used for 56.09,37.62
Meteo fallback used for 51.71,-0.48
Meteo fallback used for 35.71,139.26
Meteo fallback used for 51.23,0.13
Meteo fallback used for -0.90,36.45
Meteo fallback used for -34.06,151.15
Meteo fallback used for 35.61,140.15
Meteo fallback used for 19.22,-98.57


Meteo fallback used for -23.39,-46.65
Meteo fallback used for -33.71,151.36
Meteo fallback used for -23.69,-47.22
Meteo fallback used for -1.20,36.74
Meteo fallback used for 34.37,-117.52
Meteo fallback used for -1.04,37.31
Meteo fallback used for -23.35,-46.04
Meteo fallback used for 55.37,37.42
Meteo fallback used for 55.93,38.20
Meteo fallback used for 49.15,2.70
Meteo fallback used for 19.69,-99.50
Meteo fallback used for 34.28,-117.45
Meteo fallback used for 55.38,37.43
Meteo fallback used for 38.63,-77.34
Meteo fallback used for 51.43,-0.64
Meteo fallback used for 38.59,-77.20
Meteo fallback used for 34.08,-118.56
Meteo fallback used for 19.60,-99.70
Meteo fallback used for 49.17,1.78
Meteo fallback used for 48.73,2.77
Meteo fallback used for 39.27,-77.49
Meteo fallback used for 51.67,-0.10
Meteo fallback used for 35.36,140.00
Meteo fallback used for -33.95,150.65
Meteo fallback used for 35.35,139.55
Meteo fallback used for 51.78,-0.71
Meteo fallback used for 35.21,139.50
Meteo f

...

Meteo fallback used for 39.08,-77.00
Meteo fallback used for 39.22,-76.75
Meteo fallback used for 38.71,-77.07
Meteo fallback used for 48.45,2.71
Meteo fallback used for 51.19,0.10
Meteo fallback used for -33.64,151.24
Meteo fallback used for 35.87,139.86
Meteo fallback used for -1.40,36.93
Meteo fallback used for -33.53,151.09
Meteo fallback used for -33.73,151.42
Meteo fallback used for -23.36,-47.01
Meteo fallback used for 19.70,-99.30
Meteo fallback used for 56.14,37.79
Meteo fallback used for 55.49,37.08
Meteo fallback used for 33.98,-118.48
Meteo fallback used for 56.14,37.88
Meteo fallback used for -1.53,36.69
Meteo fallback used for 34.14,-117.87
Meteo fallback used for 19.64,-99.39
Meteo fallback used for 34.09,-117.68
Meteo fallback used for -23.17,-47.15
Meteo fallback used for 19.51,-99.65
Meteo fallback used for 48.64,2.94
Meteo fallback used for 35.41,139.61
Meteo fallback used for 34.03,-118.58
Meteo fallback used for 51.75,0.13
Meteo fallback used for -23.22,-46.15
Mete

Meteo fallback used for 55.92,38.20
Meteo fallback used for -1.24,36.99
Meteo fallback used for -23.44,-46.24
Meteo fallback used for 51.25,0.42
Meteo fallback used for 19.76,-98.82
Meteo fallback used for 34.25,-117.90
Meteo fallback used for -23.88,-46.27
Meteo fallback used for 35.39,140.09
Meteo fallback used for -34.26,150.97
Meteo fallback used for 48.43,2.16
Meteo fallback used for 48.89,2.80
Meteo fallback used for 19.55,-98.95
Meteo fallback used for -33.55,150.93
Meteo fallback used for 56.01,38.18
Meteo fallback used for -1.39,36.65
Meteo fallback used for 38.79,-76.87
Meteo fallback used for 35.62,139.56
Meteo fallback used for -33.85,151.18
Meteo fallback used for 51.89,0.09
Meteo fallback used for 34.26,-118.33
Meteo fallback used for 49.07,2.46
Meteo fallback used for 19.34,-99.12
Meteo fallback used for 51.11,-0.48
Meteo fallback used for 39.27,-76.86
Meteo fallback used for -23.70,-46.79
Meteo fallback used for 35.76,139.47
Meteo fallback used for 55.41,37.11
Meteo fal

Meteo fallback used for -33.63,151.27
Meteo fallback used for 48.97,2.01
Meteo fallback used for 39.23,-77.04
Meteo fallback used for 19.50,-98.56
Meteo fallback used for -1.56,36.33
Meteo fallback used for 56.15,38.06
Meteo fallback used for 35.28,139.85
Meteo fallback used for -23.41,-46.26
Meteo fallback used for 51.35,0.28
Meteo fallback used for 19.69,-98.62
Meteo fallback used for -34.13,151.33
Meteo fallback used for -1.41,36.71
Meteo fallback used for 34.34,-117.56
Meteo fallback used for -23.40,-47.03
Meteo fallback used for 49.14,2.60
Meteo fallback used for 39.04,-77.49
Meteo fallback used for 55.55,38.14
Meteo fallback used for 51.21,-0.65
Meteo fallback used for 34.00,-117.52
Meteo fallback used for 35.40,139.75
Meteo fallback used for -33.75,151.66
Meteo fallback used for 35.84,139.15
Meteo fallback used for 48.69,2.18
Meteo fallback used for 51.73,-0.36
Meteo fallback used for -1.66,36.66
Meteo fallback used for -33.97,151.53
Meteo fallback used for 38.76,-77.53
Meteo fa

Meteo fallback used for 19.73,-99.73
Meteo fallback used for -33.99,151.12
Meteo fallback used for 49.10,2.71
Meteo fallback used for 34.50,-118.41
Meteo fallback used for 38.57,-77.18
Meteo fallback used for 51.73,-0.12
Meteo fallback used for 55.83,37.82
Meteo fallback used for 19.43,-98.85
Meteo fallback used for 35.64,139.55
Meteo fallback used for 55.89,38.20
Meteo fallback used for -23.29,-46.67
Meteo fallback used for -1.28,36.83
Meteo fallback used for -33.55,151.59
Meteo fallback used for 33.98,-117.53
Meteo fallback used for -23.38,-46.77
Meteo fallback used for 19.14,-99.30
Meteo fallback used for 48.98,2.21
Meteo fallback used for -1.13,37.38
Meteo fallback used for 56.09,37.19
Meteo fallback used for 39.20,-76.99
Meteo fallback used for 51.45,-0.58
Meteo fallback used for 35.50,139.20
Meteo fallback used for -0.96,36.47
Meteo fallback used for 55.40,37.15
Meteo fallback used for -23.22,-46.15
Meteo fallback used for -33.84,150.72
Meteo fallback used for 19.31,-98.90
Meteo 

,tile_id,lat,lon,elev_mean,ndvi_mean,rain16,soil_moisture,risk,rgb_path,ndvi_path,srtm_path
0,tile_000004,35.358428,139.301610,20.318244,0.800943,0.0,0.240283,0.060491,tiles/tile_000004.png,ndvi/tile_000004_ndvi.png,meta/tile_000004_srtm.png
1,tile_000002,48.931859,2.184177,44.254797,0.746325,0.0,0.223897,0.078751,tiles/tile_000002.png,ndvi/tile_000002_ndvi.png,meta/tile_000002_srtm.png
2,tile_000001,38.994852,-76.662485,50.747501,0.541022,0.0,0.162307,0.153793,tiles/tile_000001.png,ndvi/tile_000001_ndvi.png,meta/tile_000001_srtm.png
3,tile_000005,-33.678879,151.533518,0.000000,0.954024,0.0,0.286207,0.013127,tiles/tile_000005.png,ndvi/tile_000005_ndvi.png,meta/tile_000005_srtm.png
4,tile_000000,34.220951,-117.961984,1274.918141,0.725389,0.0,0.217617,0.085940,tiles/tile_000000.png,ndvi/tile_000000_ndvi.png,meta/tile_000000_srtm.png


In [23]:
from huggingface_hub import HfApi, login
from pathlib import Path
login()

# login(token="hf_XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
api = HfApi()

repo_id = "Pingsz/hack-nation-global-risk-tiles"

print("Uploading large folder... This may take 5-15 minutes (800MB+)")

api.upload_large_folder(
    folder_path="/content/data",
    repo_id=repo_id,
    repo_type="dataset",
    ignore_patterns=["*.tmp", "__pycache__"],
    # num_threads=4,
    # chunk_size_mb=50,
)

print(f"🎉 SUCCESS! Your dataset is live:")
print(f"https://huggingface.co/datasets/{repo_id}")
print(f"Direct download: https://huggingface.co/datasets/{repo_id}/resolve/main/data.zip")

Failed to get upload mode: 429 Client Error: Too Many Requests for url: https://huggingface.co/api/datasets/Pingsz/hack-nation-global-risk-tiles/preupload/main (Request ID: Root=1-69103381-08a12bbe7f4530db67c0f761;74906d28-6e6b-4e65-994f-f051d3307762)

We had to rate limit you, you hit the quota of 1000 api requests per 5 minutes period. Upgrade to a PRO user or Team/Enterprise organization account (https://hf.co/pricing) to get higher limits. See https://huggingface.co/docs/hub/rate-limits
ERROR:huggingface_hub._upload_large_folder:Failed to get upload mode: 429 Client Error: Too Many Requests for url: https://huggingface.co/api/datasets/Pingsz/hack-nation-global-risk-tiles/preupload/main (Request ID: Root=1-69103381-08a12bbe7f4530db67c0f761;74906d28-6e6b-4e65-994f-f051d3307762)

We had to rate limit you, you hit the quota of 1000 api requests per 5 minutes period. Upgrade to a PRO user or Team/Enterprise organization account (https://hf.co/pricing) to get higher limits. See https://h

Uploading large folder... This may take 5-15 minutes (800MB+)


HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://huggingface.co/api/repos/create (Request ID: Root=1-69103381-58277c815f67e2682126dfd6;858b0bb4-064a-4896-b41e-06ab5271d36e)

We had to rate limit you, you hit the quota of 1000 api requests per 5 minutes period. Upgrade to a PRO user or Team/Enterprise organization account (https://hf.co/pricing) to get higher limits. See https://huggingface.co/docs/hub/rate-limits

In [27]:
! df -h /content/data

Failed to get upload mode: 429 Client Error: Too Many Requests for url: https://huggingface.co/api/datasets/Pingsz/hack-nation-global-risk-tiles/preupload/main (Request ID: Root=1-691033bc-568582a163a03d4851dc7073;fc4b2a06-f823-496b-8162-8f251dabe3d8)

We had to rate limit you, you hit the quota of 1000 api requests per 5 minutes period. Upgrade to a PRO user or Team/Enterprise organization account (https://hf.co/pricing) to get higher limits. See https://huggingface.co/docs/hub/rate-limits
ERROR:huggingface_hub._upload_large_folder:Failed to get upload mode: 429 Client Error: Too Many Requests for url: https://huggingface.co/api/datasets/Pingsz/hack-nation-global-risk-tiles/preupload/main (Request ID: Root=1-691033bc-568582a163a03d4851dc7073;fc4b2a06-f823-496b-8162-8f251dabe3d8)

We had to rate limit you, you hit the quota of 1000 api requests per 5 minutes period. Upgrade to a PRO user or Team/Enterprise organization account (https://hf.co/pricing) to get higher limits. See https://h

Filesystem      Size  Used Avail Use% Mounted on
overlay         108G   40G   69G  37% /


In [29]:
!pip install huggingface_hub[cli] git-lfs
!git lfs install

Failed to get upload mode: 429 Client Error: Too Many Requests for url: https://huggingface.co/api/datasets/Pingsz/hack-nation-global-risk-tiles/preupload/main (Request ID: Root=1-69103419-5199359e544ceb390b48a5d8;4a6bf834-81f9-4209-948c-437a517d832e)

We had to rate limit you, you hit the quota of 1000 api requests per 5 minutes period. Upgrade to a PRO user or Team/Enterprise organization account (https://hf.co/pricing) to get higher limits. See https://huggingface.co/docs/hub/rate-limits
ERROR:huggingface_hub._upload_large_folder:Failed to get upload mode: 429 Client Error: Too Many Requests for url: https://huggingface.co/api/datasets/Pingsz/hack-nation-global-risk-tiles/preupload/main (Request ID: Root=1-69103419-5199359e544ceb390b48a5d8;4a6bf834-81f9-4209-948c-437a517d832e)

We had to rate limit you, you hit the quota of 1000 api requests per 5 minutes period. Upgrade to a PRO user or Team/Enterprise organization account (https://hf.co/pricing) to get higher limits. See https://h

Failed to get upload mode: 429 Client Error: Too Many Requests for url: https://huggingface.co/api/datasets/Pingsz/hack-nation-global-risk-tiles/preupload/main (Request ID: Root=1-6910341a-589eb0b95491bdc0588380e9;a2359664-90d7-499f-85fa-36d5a877ecc5)

We had to rate limit you, you hit the quota of 1000 api requests per 5 minutes period. Upgrade to a PRO user or Team/Enterprise organization account (https://hf.co/pricing) to get higher limits. See https://huggingface.co/docs/hub/rate-limits
ERROR:huggingface_hub._upload_large_folder:Failed to get upload mode: 429 Client Error: Too Many Requests for url: https://huggingface.co/api/datasets/Pingsz/hack-nation-global-risk-tiles/preupload/main (Request ID: Root=1-6910341a-589eb0b95491bdc0588380e9;a2359664-90d7-499f-85fa-36d5a877ecc5)

We had to rate limit you, you hit the quota of 1000 api requests per 5 minutes period. Upgrade to a PRO user or Team/Enterprise organization account (https://hf.co/pricing) to get higher limits. See https://h

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/67.7 kB ? eta -:--:--

Failed to get upload mode: 429 Client Error: Too Many Requests for url: https://huggingface.co/api/datasets/Pingsz/hack-nation-global-risk-tiles/preupload/main (Request ID: Root=1-6910341a-7998b6fb1942ef7c043a4f88;157b5733-7a46-4a8d-a293-a7daeb371ca7)

We had to rate limit you, you hit the quota of 1000 api requests per 5 minutes period. Upgrade to a PRO user or Team/Enterprise organization account (https://hf.co/pricing) to get higher limits. See https://huggingface.co/docs/hub/rate-limits
ERROR:huggingface_hub._upload_large_folder:Failed to get upload mode: 429 Client Error: Too Many Requests for url: https://huggingface.co/api/datasets/Pingsz/hack-nation-global-risk-tiles/preupload/main (Request ID: Root=1-6910341a-7998b6fb1942ef7c043a4f88;157b5733-7a46-4a8d-a293-a7daeb371ca7)

We had to rate limit you, you hit the quota of 1000 api requests per 5 minutes period. Upgrade to a PRO user or Team/Enterprise organization account (https://hf.co/pricing) to get higher limits. See https://h

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 2.4 MB/s eta 0:00:00


Failed to get upload mode: 429 Client Error: Too Many Requests for url: https://huggingface.co/api/datasets/Pingsz/hack-nation-global-risk-tiles/preupload/main (Request ID: Root=1-6910341b-476c66e0271095d55e5401eb;496bd11b-d92f-4e95-aa36-2f2ebee77154)

We had to rate limit you, you hit the quota of 1000 api requests per 5 minutes period. Upgrade to a PRO user or Team/Enterprise organization account (https://hf.co/pricing) to get higher limits. See https://huggingface.co/docs/hub/rate-limits
ERROR:huggingface_hub._upload_large_folder:Failed to get upload mode: 429 Client Error: Too Many Requests for url: https://huggingface.co/api/datasets/Pingsz/hack-nation-global-risk-tiles/preupload/main (Request ID: Root=1-6910341b-476c66e0271095d55e5401eb;496bd11b-d92f-4e95-aa36-2f2ebee77154)

We had to rate limit you, you hit the quota of 1000 api requests per 5 minutes period. Upgrade to a PRO user or Team/Enterprise organization account (https://hf.co/pricing) to get higher limits. See https://h

Failed to get upload mode: 429 Client Error: Too Many Requests for url: https://huggingface.co/api/datasets/Pingsz/hack-nation-global-risk-tiles/preupload/main (Request ID: Root=1-6910341d-627d2c511e6d55420b9b9579;59e1c092-a0f6-41f0-a675-1af7853b958d)

We had to rate limit you, you hit the quota of 1000 api requests per 5 minutes period. Upgrade to a PRO user or Team/Enterprise organization account (https://hf.co/pricing) to get higher limits. See https://huggingface.co/docs/hub/rate-limits
ERROR:huggingface_hub._upload_large_folder:Failed to get upload mode: 429 Client Error: Too Many Requests for url: https://huggingface.co/api/datasets/Pingsz/hack-nation-global-risk-tiles/preupload/main (Request ID: Root=1-6910341d-627d2c511e6d55420b9b9579;59e1c092-a0f6-41f0-a675-1af7853b958d)

We had to rate limit you, you hit the quota of 1000 api requests per 5 minutes period. Upgrade to a PRO user or Team/Enterprise organization account (https://hf.co/pricing) to get higher limits. See https://h

Git LFS initialized.


In [30]:
!huggingface-cli login

Failed to get upload mode: (Request ID: Root=1-69103435-4580d7f613e8a5903d006581;b81c3f44-155e-4dcc-b827-5b4b10a6613b)

403 Forbidden: Forbidden: you must use a write token to upload to a repository..
Cannot access content at: https://huggingface.co/api/datasets/Pingsz/hack-nation-global-risk-tiles/preupload/main.
Make sure your token has the correct permissions.
ERROR:huggingface_hub._upload_large_folder:Failed to get upload mode: (Request ID: Root=1-69103435-4580d7f613e8a5903d006581;b81c3f44-155e-4dcc-b827-5b4b10a6613b)

403 Forbidden: Forbidden: you must use a write token to upload to a repository..
Cannot access content at: https://huggingface.co/api/datasets/Pingsz/hack-nation-global-risk-tiles/preupload/main.
Make sure your token has the correct permissions.
Failed to get upload mode: (Request ID: Root=1-69103435-36a7a8b55303e61b5e9b8c1b;2a648481-7488-4361-a45c-752d8719a1aa)

403 Forbidden: Forbidden: you must use a write token to upload to a repository..
Cannot access content at

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `hf auth whoami` to get more information or `hf auth logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 

Failed to get upload mode: (Request ID: Root=1-69103436-0a232bd94c819f3711a3d30f;e8288a6f-c546-437f-93a1-a0fa9282d736)

403 Forbidden: Forbidden: you must use a write token to upload to a repository..
Cannot access content at: https://huggingface.co/api/datasets/Pingsz/hack-nation-global-risk-tiles/preupload/main.
Make sure your token has the correct permissions.
ERROR:huggingface_hub._upload_large_folder:Failed to get upload mode: (Request ID: Root=1-69103436-0a232bd94c819f3711a3d30f;e8288a6f-c546-437f-93a1-a0fa9282d736)

403 Forbidden: Forbidden: you must use a write token to upload to a repository..
Cannot access content at: https://huggingface.co/api/datasets/Pingsz/hack-nation-global-risk-tiles/preupload/main.
Make sure your token has the correct permissions.
Failed to get upload mode: (Request ID: Root=1-69103436-2f26ea1d23d477e8215f9674;a6034b7a-ecd2-4ae6-b571-00988062a5fc)

403 Forbidden: Forbidden: you must use a write token to upload to a repository..
Cannot access content at

Traceback (most recent call last):
object address  : 0x7b0f6586a440
object refcount : 3
object type     : 0xa2a4e0
object type name: KeyboardInterrupt
object repr     : KeyboardInterrupt()
lost sys.stderr
^C


In [32]:
!zip -r -q /content/global-risk-tiles-dataset.zip /content/data

Failed to get upload mode: 429 Client Error: Too Many Requests for url: https://huggingface.co/api/datasets/Pingsz/hack-nation-global-risk-tiles/preupload/main (Request ID: Root=1-69103476-2f39afba14117c4e2cec1f06;8bb4959a-48e5-4e7b-8303-50564cbcdb7b)

We had to rate limit you, you hit the quota of 1000 api requests per 5 minutes period. Upgrade to a PRO user or Team/Enterprise organization account (https://hf.co/pricing) to get higher limits. See https://huggingface.co/docs/hub/rate-limits
ERROR:huggingface_hub._upload_large_folder:Failed to get upload mode: 429 Client Error: Too Many Requests for url: https://huggingface.co/api/datasets/Pingsz/hack-nation-global-risk-tiles/preupload/main (Request ID: Root=1-69103476-2f39afba14117c4e2cec1f06;8bb4959a-48e5-4e7b-8303-50564cbcdb7b)

We had to rate limit you, you hit the quota of 1000 api requests per 5 minutes period. Upgrade to a PRO user or Team/Enterprise organization account (https://hf.co/pricing) to get higher limits. See https://h

In [2]:
! pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 25.4 MB/s eta 0:00:00


In [4]:
import os
import requests
import rasterio
import numpy as np
import pandas as pd
from tqdm import tqdm

tif_url = "https://sedac.ciesin.columbia.edu/downloads/data/global-multi-hazard-risk-distribution/global-multi-hazard-risk-distribution_2015.tif"
local_tif = "/content/global_mhazard_2015.tif"

if not os.path.exists(local_tif):
    print("📥 Downloading NASA SEDAC Multi-Hazard Risk Index (2015)...")
    r = requests.get(tif_url, stream=True)
    r.raise_for_status()
    with open(local_tif, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    print("✅ Download complete:", local_tif)
else:
    print("✅ Using cached:", local_tif)

labels_path = "/content/data/labels.csv"
assert os.path.exists(labels_path), "labels.csv not found — run build_dataset() first."
df = pd.read_csv(labels_path)

with rasterio.open(local_tif) as src:
    coords = [(lon, lat) for lat, lon in zip(df["lat"], df["lon"])]
    sampled_vals = []
    for xy in tqdm(coords, desc="Sampling hazard risk"):
        try:
            val = list(src.sample([xy]))[0][0]
        except Exception:
            val = np.nan
        sampled_vals.append(val)

df["risk"] = np.nan_to_num(sampled_vals, nan=0.0)
df["risk"] = df["risk"].astype(float)
df["risk"] = (df["risk"] - df["risk"].min()) / (df["risk"].max() - df["risk"].min())
df["risk"] = df["risk"].clip(0, 1)

print("\n📊 Risk Value Distribution:")
print(df["risk"].describe())

df.to_csv(labels_path, index=False)
print(f"\n✅ Updated dataset saved → {labels_path}")

📥 Downloading NASA SEDAC Multi-Hazard Risk Index (2015)...


ConnectTimeout: HTTPSConnectionPool(host='sedac.ciesin.columbia.edu', port=443): Max retries exceeded with url: /downloads/data/global-multi-hazard-risk-distribution/global-multi-hazard-risk-distribution_2015.tif (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7a532f9b74d0>, 'Connection to sedac.ciesin.columbia.edu timed out. (connect timeout=None)'))

In [ ]:
import ee
import pandas as pd
import numpy as np
from tqdm import tqdm
import time



labels_path = "/content/data/labels.csv"
df = pd.read_csv(labels_path)

risk_img = ee.Image("UNDRR/INFORM/INFORM_2023_Risk")

print("Available bands:", risk_img.bandNames().getInfo())

# For Sampleing the 'Risk' band globally
values = []
band_name = "Risk"  #need to revisit
for lat, lon in tqdm(zip(df["lat"], df["lon"]), total=len(df), desc="Sampling INFORM Risk"):
    try:
        pt = ee.Geometry.Point([float(lon), float(lat)])
        val = risk_img.select(band_name).reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=pt,
            scale=10000,
            maxPixels=1e9
        ).get(band_name).getInfo()
        values.append(float(val) if val is not None else np.nan)
    except Exception as e:
        values.append(np.nan)
        time.sleep(0.1)  # retry pacing

df["risk"] = np.nan_to_num(values, nan=0.0)
# Normalize 0–1
if df["risk"].max() > df["risk"].min():
    df["risk"] = (df["risk"] - df["risk"].min()) / (df["risk"].max() - df["risk"].min())
df["risk"] = df["risk"].clip(0, 1)

print("\n✅ INFORM Risk sampling complete.")
print(df["risk"].describe())

df.to_csv(labels_path, index=False)
print(f"✅ Updated dataset saved to {labels_path}")


EEException: Image.load: Image asset 'UNDRR/INFORM/INFORM_2023_Risk' not found (does not exist or caller does not have access).

In [ ]:
##### Connection Break
import os, requests, rasterio, numpy as np, pandas as pd
from tqdm import tqdm

os.makedirs("/content/hazard_rasters", exist_ok=True)

srcs = {
    # NASA SEDAC: GPWv4 population density (proxy for exposure)
    "pop": "https://sedac.ciesin.columbia.edu/downloads/data/gpw-v4/gpw-v4-population-density-adjusted-to-2015-unwpp-country-totals-rev11/gpw-v4-population-density-adjusted-to-2015-unwpp-country-totals-rev11_2020_30_sec_tif.zip",
    # NASA FIRMS: burned-area climatology raster
    "burn": "https://storage.googleapis.com/gfw-data-lake/global_fire_emissions_2020_mean.tif"
}

def download_and_unzip(name, url, dest_dir):
    path_zip = f"{dest_dir}/{name}.zip"
    path_tif = None
    print(f"📥 downloading {name}")
    r = requests.get(url, stream=True, timeout=300)
    r.raise_for_status()
    with open(path_zip, "wb") as f:
        for ch in r.iter_content(8192): f.write(ch)
    if url.endswith(".zip"):
        import zipfile
        with zipfile.ZipFile(path_zip, "r") as z:
            for fn in z.namelist():
                if fn.endswith(".tif"):
                    z.extract(fn, dest_dir)
                    path_tif = os.path.join(dest_dir, fn)
                    break
    else:
        path_tif = path_zip
    return path_tif

paths = {}
for k,u in srcs.items():
    tif_path = f"/content/hazard_rasters/{k}.tif"
    if not os.path.exists(tif_path):
        p = download_and_unzip(k,u,"/content/hazard_rasters")
        if p and not p.endswith(".tif"):
            tif_path = [x for x in os.listdir("/content/hazard_rasters") if x.endswith(".tif")][0]
            tif_path = os.path.join("/content/hazard_rasters", tif_path)
        else:
            tif_path = p
    paths[k] = tif_path

def safe_open(fp):
    try: return rasterio.open(fp)
    except Exception as e:
        print("⚠️ cannot open", fp, e)
        return None

r_pop = safe_open(paths["pop"])
r_burn = safe_open(paths["burn"])

def sample(lat,lon):
    vals=[]
    for ds in [r_pop,r_burn]:
        if ds is None: continue
        if not (ds.bounds.left<lon<ds.bounds.right and ds.bounds.bottom<lat<ds.bounds.top):
            continue
        try:
            v=list(ds.sample([(lon,lat)]))[0][0]
            if np.isfinite(v): vals.append(v)
        except: pass
    if not vals: return np.nan
    return float(np.clip(np.log1p(np.mean(vals))/10,0,1))

df=pd.read_csv("/content/data/labels.csv")
df["risk"]=[sample(lat,lon) for lat,lon in tqdm(zip(df.lat,df.lon),total=len(df))]
df["risk"].replace([np.inf,-np.inf],np.nan,inplace=True)
df["risk"].fillna(df["risk"].mean(),inplace=True)
df["risk"]=df["risk"].clip(0,1)

print(df["risk"].describe())
df.to_csv("/content/data/labels.csv",index=False)
print("✅ Dataset ready for training.")


📥 downloading pop


ConnectTimeout: HTTPSConnectionPool(host='sedac.ciesin.columbia.edu', port=443): Max retries exceeded with url: /downloads/data/gpw-v4/gpw-v4-population-density-adjusted-to-2015-unwpp-country-totals-rev11/gpw-v4-population-density-adjusted-to-2015-unwpp-country-totals-rev11_2020_30_sec_tif.zip (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7a53289b78c0>, 'Connection to sedac.ciesin.columbia.edu timed out. (connect timeout=300)'))

In [21]:
import pandas as pd, numpy as np
from tqdm import tqdm

coords = np.random.uniform(low=[-180, -60], high=[180, 80], size=(1200, 2))
pts = [ee.Geometry.Point(float(lon), float(lat)) for lon, lat in coords]

modis = ee.ImageCollection("MODIS/061/MCD64A1").select("BurnDate").mean()
srtm = ee.Image("USGS/SRTMGL1_003").select("elevation")
smap = ee.ImageCollection("NASA/SMAP/SPL4SMGP/008").select("ssm").mean()
gsw = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("occurrence")
modis_ndvi = ee.ImageCollection("MODIS/061/MOD13A2").select("NDVI").mean()
stack = ee.Image.cat([modis, srtm, smap, gsw, modis_ndvi])

vals = []
for p in tqdm(pts):
    v = stack.sample(region=p, scale=1000).first()
    v = v.getInfo()
    if v and 'properties' in v:
        vals.append(v['properties'])
df = pd.DataFrame(vals).dropna()
df["lon"] = [c['coordinates'][0] for c in coords[:len(df)]]
df["lat"] = [c['coordinates'][1] for c in coords[:len(df)]]
df["risk"] = (df.mean(axis=1) - df.mean(axis=1).min()) / (df.mean(axis=1).max() - df.mean(axis=1).min())
df.describe()
df.to_csv("/content/geo_risk_dataset.csv", index=False)
print("✅ dataset saved to /content/geo_risk_dataset.csv")


  0%|          | 0/1200 [00:00<?, ?it/s]


EEException: reduce.mean: Error in map(ID=20150331_0130):
Image.select: Band pattern 'ssm' did not match any bands. Available bands: [sm_surface, sm_rootzone, sm_profile, sm_surface_wetness, sm_rootzone_wetness, sm_profile_wetness, surface_temp, soil_temp_layer1, soil_temp_layer2, soil_temp_layer3, soil_temp_layer4, soil_temp_layer5, soil_temp_layer6, snow_mass, snow_depth, land_evapotranspiration_flux, overland_runoff_flux, baseflow_flux, snow_melt_flux, soil_water_infiltration_flux, land_fraction_saturated, land_fraction_unsaturated, land_fraction_wilting, land_fraction_snow_covered, heat_flux_sensible, heat_flux_latent, heat_flux_ground, net_downward_shortwave_flux, net_downward_longwave_flux, radiation_shortwave_downward_flux, radiation_longwave_absorbed_flux, precipitation_total_surface_flux, snowfall_surface_flux, surface_pressure, height_lowatmmodlay, temp_lowatmmodlay, specific_humidity_lowatmmodlay, windspeed_lowatmmodlay, vegetation_greenness_fraction, leaf_area_index, sm_roo

In [26]:
import os, ee, requests, numpy as np, pandas as pd
from tqdm import tqdm
from PIL import Image
from io import BytesIO

os.makedirs("/content/data/tiles", exist_ok=True)
os.makedirs("/content/data/ndvi", exist_ok=True)
os.makedirs("/content/data/meta", exist_ok=True)

coords = np.random.uniform(low=[-180, -60], high=[180, 80], size=(1200, 2))
pts = [ee.Geometry.Point(float(lon), float(lat)) for lon, lat in coords]

modis = ee.ImageCollection("MODIS/061/MCD64A1").select("BurnDate").mean()
srtm = ee.Image("USGS/SRTMGL1_003").select("elevation")
smap = ee.ImageCollection("NASA/SMAP/SPL4SMGP/008").select("sm_surface").mean()
gsw = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("occurrence")
modis_ndvi = ee.ImageCollection("MODIS/061/MOD13A2").select("NDVI").mean()
stack = ee.Image.cat([modis, srtm, smap, gsw, modis_ndvi])

vals, latlons = [], []
for i, (lon, lat) in enumerate(tqdm(coords)):
    p = ee.Geometry.Point(float(lon), float(lat))
    v = stack.sample(region=p, scale=1000).first()
    v = v.getInfo()
    if not v or "properties" not in v:
        continue
    vals.append(v["properties"])
    latlons.append((lat, lon))
    region = p.buffer(2000).bounds()
    rgb = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterBounds(p).filterDate("2023-01-01", "2024-01-01").sort("CLOUDY_PIXEL_PERCENTAGE").first()
    if rgb:
        rgb_viz = rgb.select(["B4", "B3", "B2"]).visualize(min=0, max=3000)
        ndvi = rgb.normalizedDifference(["B8", "B4"]).rename("NDVI")
        elev = srtm.visualize(min=0, max=3000)
        urls = {
            "rgb": rgb_viz.getThumbURL({"region": region, "dimensions": "128x128", "format": "png"}),
            "ndvi": ndvi.getThumbURL({"region": region, "dimensions": "128x128", "min": -0.5, "max": 1, "palette": ["white", "green"], "format": "png"}),
            "elev": elev.getThumbURL({"region": region, "dimensions": "128x128", "format": "png"})
        }
        for k, u in urls.items():
            try:
                r = requests.get(u, timeout=30)
                if r.status_code == 200:
                    Image.open(BytesIO(r.content)).save(f"/content/data/{k}/{i:06d}_{k}.png")
            except:
                pass

df = pd.DataFrame(vals).dropna()
df["lat"], df["lon"] = zip(*latlons[:len(df)])
df["risk"] = (df.mean(axis=1) - df.mean(axis=1).min()) / (df.mean(axis=1).max() - df.mean(axis=1).min())
df.to_csv("/content/data/geo_risk_dataset.csv", index=False)
print("✅ dataset and image tiles saved under /content/data/")


  0%|          | 0/1200 [00:24<?, ?it/s]


EEException: User memory limit exceeded.

In [23]:
!rm -rf /content/data

In [25]:
!rehash

/bin/bash: line 1: rehash: command not found


In [27]:
import os, ee, numpy as np, pandas as pd
from tqdm import tqdm

os.makedirs("/content/data", exist_ok=True)

coords = np.random.uniform(low=[-180, -60], high=[180, 80], size=(1200, 2))
features = [ee.Feature(ee.Geometry.Point(lon, lat)) for lon, lat in coords]
fc = ee.FeatureCollection(features)

modis = ee.ImageCollection("MODIS/061/MCD64A1").select("BurnDate").mean()
srtm = ee.Image("USGS/SRTMGL1_003").select("elevation")
smap = ee.ImageCollection("NASA/SMAP/SPL4SMGP/008").select("sm_surface").mean()
gsw = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("occurrence")
modis_ndvi = ee.ImageCollection("MODIS/061/MOD13A2").select("NDVI").mean()

stack = ee.Image.cat([modis, srtm, smap, gsw, modis_ndvi])

sampled = stack.sampleRegions(collection=fc, scale=1000)
vals = sampled.limit(1200).getInfo()

rows = []
for f in tqdm(vals["features"]):
    p = f["properties"]
    if all(k in p for k in ["BurnDate","elevation","sm_surface","occurrence","NDVI"]):
        rows.append(p)

df = pd.DataFrame(rows).dropna()
df["risk"] = (df.mean(axis=1)-df.mean(axis=1).min())/(df.mean(axis=1).max()-df.mean(axis=1).min())
df.to_csv("/content/data/geo_risk_dataset.csv", index=False)
print("✅ saved /content/data/geo_risk_dataset.csv")
print(df.describe())


EEException: User memory limit exceeded.

In [28]:
import ee, numpy as np, pandas as pd, requests

coords = np.random.uniform(low=[-180, -60], high=[180, 80], size=(1200, 2))
features = [ee.Feature(ee.Geometry.Point(lon, lat)) for lon, lat in coords]
fc = ee.FeatureCollection(features)

modis = ee.ImageCollection("MODIS/061/MCD64A1").select("BurnDate").mean()
srtm = ee.Image("USGS/SRTMGL1_003").select("elevation")
smap = ee.ImageCollection("NASA/SMAP/SPL4SMGP/008").select("sm_surface").mean()
gsw = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("occurrence")
modis_ndvi = ee.ImageCollection("MODIS/061/MOD13A2").select("NDVI").mean()

stack = ee.Image.cat([modis, srtm, smap, gsw, modis_ndvi])
sampled = stack.sampleRegions(collection=fc, scale=1000).limit(1200)

url = sampled.getDownloadURL("csv")
r = requests.get(url)
open("/content/geo_risk_dataset.csv","wb").write(r.content)

df = pd.read_csv("/content/geo_risk_dataset.csv").dropna()
df["risk"] = (df.mean(axis=1)-df.mean(axis=1).min())/(df.mean(axis=1).max()-df.mean(axis=1).min())
df.to_csv("/content/geo_risk_dataset.csv", index=False)
print("✅ saved /content/geo_risk_dataset.csv")
print(df.describe())


EEException: User memory limit exceeded.